# Heterogeneous mixing

By default, the Summer compartmental model assumes that each person in the model comes into contact with every other person at the same rate (homogeneous mixing). This assumption is preserved even after stratification has been applied (by default), such that the force of infection calculation will be the same for all model strata implemented. This notebook considers the incorporation of heterogeneous mixing patterns. For example, children may tend to interact more with other children than they do with elderly people (this is termed with-like or assortative mixing). When considering the impact of control strategies, especially strategies targeting sub groups of people, it is likely to be important to consider these differences in contact patterns.

## Standard preliminaries
Before we get into the code for heterogeneous mixing, let's start off with some of our standard (or "boilerplate") code to get everything set up.

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
from typing import List, Union
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import plotly.express as px

from summer import CompartmentalModel, Stratification

pd.options.plotting.backend = "plotly"

from import_phl_data import get_population_and_epi_data

In [ ]:
# The data import module lives in a file on AuTuMN github - download it for colab use
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/philippines/import_phl_data.py

import import_phl_data

In [ ]:
# Shareable google drive links
PHL_DOH_LINK = "1fFKoNVan7PS6BpBr01nwByNTLLu6z1AA"  # sheet 05 daily report.
PHL_FASSSTER_LINK = "15eDyTjXng2Zh38DVhmeNy0nQSqOMlGj3" # Fassster google drive zip file.
initial_population, df = get_population_and_epi_data(PHL_DOH_LINK, PHL_FASSSTER_LINK) 

# We define a day zero for the analysis
COVID_BASE_DATE = datetime(2019, 12, 31)

## Getting mixing matrices

In [ ]:
simple_matrix = np.array([[1., 0.5], [0.5, 1.]])
print(simple_matrix.shape)
px.imshow(simple_matrix)

Get a mixing matrix that we can use for the Philippines.
This is not actually empiric data for the Philippines, but is what we have been using
and has the right format.

In [ ]:
# Get the mixing matrix for the Philippines - currently this is actually Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

In [ ]:
# Let's have a look at the matrix using plotly express
print(age_mixing_matrix.shape)
px.imshow(age_mixing_matrix)

## Model

### Define a model

In [ ]:
def build_unstratified_model(base_compartments: List[str], parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        base_compartments: The names of the base (unstratified compartments)
        parameters: Flow parameters
    Returns:
        A compartmental model currently without age stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=base_compartments,
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    return model

### Age stratification process

In [ ]:
def get_age_stratification(
    compartments_to_stratify: List[str], 
    strata: List[str],
    matrix: Union[np.ndarray, callable],
) -> Stratification:
    """
    Create a summer stratification object that stratifies all of the compartments into
    strata, which are intended to represent age bands according to the user inputs.
    This is essentially adapting the model's age stratification approach to the format
    of the mixing matrix, which is a reasonable approach.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify, which should be all the compartments
        strata: The strata to be implemented in the age stratification
        matrix: The mixing matrix we are applying for the age structure
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    
    if isinstance(matrix, np.ndarray):
        msg = "Mixing matrix is not 2-dimensional"
        assert matrix.ndim == 2, msg

        msg = f"Dimensions of the mixing matrix incorrect: {matrix.shape[0]}, {matrix.shape[1]}, {len(strata)}"
        assert matrix.shape[0] == matrix.shape[1] == len(strata), msg
    
    # Create the stratification, just naming the age groups by their starting value
    strat = Stratification(name="age", strata=strata, compartments=compartments_to_stratify)
    
    # Add the mixing matrix to the stratification
    strat.set_mixing_matrix(matrix)

    return strat

### Combine the two processes together

In [ ]:
def build_stratified_model(
    base_compartments: List[str], 
    parameters: dict,
    strata: List[str],
    age_mixing_matrix: np.ndarray,
) -> CompartmentalModel:
    """
    Get the model object with the age stratification with heterogeneous
    mixing applied to it.
    
    Arguments:
        base_compartments: The names of the compartments to be implemented
        parameters: A dictionary containing the parameter values to use
        strata: The strata to be implemented in the age stratification
        age_mixing_matrix: 
        
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(base_compartments, parameters)

    # Get and apply the stratification
    age_strat = get_age_stratification(base_compartments, strata, age_mixing_matrix)
    model.stratify_with(age_strat)

    return model

### Actually run the models

In [ ]:
# Build and run the stratified model with some arbitrary parameters
analysis_start_date = datetime(2021,1,1)
analysis_end_date = analysis_start_date + timedelta(days=300.)
start_date_int = (analysis_start_date - COVID_BASE_DATE).days
end_date_int = (analysis_end_date - COVID_BASE_DATE).days

simple_age_params = {
    "contact_rate": 0.5,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.02,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.,
}

base_compartments = ["S", "E", "I", "R"]
strata = ["child", "adult"]

simple_age_strat_model = build_stratified_model(base_compartments, simple_age_params, strata, simple_matrix)

In [ ]:
simple_age_strat_model.run()

progressions = simple_age_strat_model.get_derived_outputs_df()["progressions"]
simple_cases_modelled = progressions * simple_age_params["reporting_fraction"]

pd.DataFrame(
    {
        "observed": df["cases"],
        "modelled": simple_cases_modelled,
    }
).plot(title="Uncalibrated first run")

## Manual calibration to notifications

In [ ]:
# Define a set of parameters for this run
full_age_params = {
        "contact_rate": 0.05,
        "progression_rate": 0.2,
        "recovery_rate": 0.2,
        "death_rate": 0.01,
        "reporting_fraction": 0.095,
        "start_time": start_date_int,
        "end_time": end_date_int,
        "infectious_seed": 100.
    }

strata = [str(i) for i in range(0, 80, 5)]

# Run using our custom parameters (recycling the base compartments list from earlier)
full_age_strat_model = build_stratified_model(base_compartments, full_age_params, strata, age_mixing_matrix)
full_age_strat_model.run()

# Get the outputs
progressions = full_age_strat_model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * full_age_params["reporting_fraction"]

pd.DataFrame(
    {
        "observed": df["cases"],
        "modelled": notifications_modelled
    }
).plot()

## Scenario analysis

In [ ]:
def get_step_matrix(time, computed_values):
    """
    Rather than getting a numpy array,
    use a function that returns an array,
    which is also permissible in summer.
    
    Args:
        These are standard arguments that must be used for mixing matrices that vary over time
    Return:
        The mixing matrix at the time of interest
    """
    
    if time < 435:
        return simple_matrix
    else:
        return np.zeros_like(simple_matrix)

In [ ]:
# Define the model running period and convert to a numeric representation
step_matrix_model = build_stratified_model(base_compartments, simple_age_params, ["child", "adult"], get_step_matrix)

In [ ]:
step_matrix_model.run()

progressions = step_matrix_model.get_derived_outputs_df()["progressions"]
notifications_modelled = progressions * simple_age_params["reporting_fraction"]

pd.DataFrame(
    {
        "observed": df["cases"],
        "baseline": simple_cases_modelled,
        "scenario": notifications_modelled,
    }
).plot()